# Similarity

---------------

__Create__: Yuan.Sh

__Date__: 2021-11-30 22:03:57

__Email__: yuansh3354@163.com

__Blog__: https://blog.csdn.net/qq_40966210

__Address__:Fujian Medical University

---------------

## Summary of data 

| Cell_subType    | Count | Cancer or Normal |  |
| --------------- | ----- | ---------------- |  |
| AT1             | 530   | Normal           |  |
| AT2             | 2020  | Normal           |  |
| Ciliated        | 654   | Normal           |  |
| Club            | 439   | Normal           |  |
| Undetermined    | 60    | NA               |  |
| tS1             | 3270  | Tumor            |  |
| tS2             | 3018  | Tumor            |  |
| tS3             | 64    | Tumor            |  |
| nan             | 1628  | NA               |  |
| Malignant cells | 24784 | Tumor            |  |

---

In the [article of Data](https://doi.org/10.1038/s41467-020-16164-1), authors use infercnv to define which cell is cancer cell. If any cell has copy number variation(use some threshold, see article), that is considered as a cancer cell, otherwise it is Na.

However, generally speaking, we always treat no-CNV cell as Normal cell. Because that the threshold is not suitable for anyother datas, and also, not everyone konws how to choose suitable threshold.

__So, in the following study, we drop-out the `NA` value, as the article label. and use `Normal` label replace `NA` as inferCNV label__

> In this section, we just only evaluated the `Similarity` between __artivle-label__ and __CaSee-label__

In [2]:
### 常规设置
import os
import math
import itertools
import warnings
import numpy as np
import glob 
import pandas as pd
import pytorch_lightning as pl
from collections import Counter
import multiprocessing
import matplotlib.pyplot as plt
from PIL import Image
import scanpy as sc
import re 
meta = pd.read_csv('../GSE131907_epi_meta.csv',index_col=0)


/home/yuansh/anaconda3/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
predict = pd.read_csv('../merge_data_h5_file/merge_data_predict_Integration_.csv',index_col=0)
meta = meta.dropna()
ids = meta.index
predict = predict.loc[ids]
sum(predict['scale_predict'] == meta['article_label']) / predict.shape[0]
Counter(predict['scale_predict'])
Counter(meta['article_label'])
from sklearn.metrics import confusion_matrix
confusion_matrix(predict['scale_predict'], meta['article_label'], labels=['Cancer','Normal'])

0.9413875254743248

Counter({'Normal': 4645, 'Cancer': 30194})

Counter({'Normal': 3703, 'Cancer': 31136})

array([[29644,   550],
       [ 1492,  3153]])

##### Calculate tissue similarity

In [3]:
# meta['predict'] = predict['scale_predict']
# df = meta.groupby(['Sample_Origin','article_label','predict']).count()
# df = pd.DataFrame(df['Cell_subtype'])
# df.columns = ['Count']
# df = df.loc[Counter(meta['Sample_Origin']).keys()]
# ids = [val for val in Counter(meta['Sample_Origin']).values() for i in range(2)]
# 
# df['similarity']  = df['Count'] / ids
# df.to_csv('~/Desktop/GSE131907_tiss_similarity.csv')
# 
# df

##### PE

In [4]:
# meta = pd.read_csv('../GSE131907_epi_meta.csv',index_col=0)
# predict = pd.read_csv('../merge_data_h5_file/merge_data_predict_Integration_.csv',index_col=0)
# ids = meta.index.intersection(predict.index)
# meta = meta.loc[ids,]
# predict = predict.loc[ids,]
# 
# meta['predict'] = predict['scale_predict']
# df = meta.groupby(['Sample_Origin','article_label','predict']).count()
# df
# df = pd.DataFrame(df['Barcode'])
# df.columns = ['Count']
# df = df.loc[Counter(meta['Sample_Origin']).keys()]
# ids = [val for val in Counter(meta['Sample_Origin']).values() for i in range(2)]
# df  = df.loc[[i for i in df.index if i[1] == i[2]]] 
# 
# 
#  meta.groupby(['Sample_Origin']).count()
# 
#  meta.groupby(['Sample_Origin','cnv.label']).count()
# 
#  meta.groupby(['Sample_Origin','predict']).count()
# 
# meta = pd.read_table('../GSE131907_Lung_Cancer_cell_annotation.txt',index_col=0)
# 
# pd.DataFrame(meta.groupby(['Sample_Origin','Cell_type']).count()['Barcode']).to_csv('/home/yuansh/Desktop/1.csv')
# 
# meta[['Sample_Origin','Cell_type']].to_csv('/home/yuansh/Desktop/1.csv')

##### immune cell cor Cancer 

In [5]:
info = pd.read_table('../GSE131907_Lung_Cancer_cell_annotation.txt',index_col=0)
meta['predict'] = predict['scale_predict']
ids1 = list(Counter(meta['Sample_Origin']).keys())
ids2 = list(Counter(info['Cell_type']).keys())

df1 = meta.groupby(['Sample_Origin','predict']).count().loc[Counter(meta['Sample_Origin']).keys()]
df2 = info.groupby(['Sample_Origin','Cell_type']).count().loc[Counter(meta['Sample_Origin']).keys()]
ids = [(i,'Cancer')for i in ids1]

c1 =  df1.loc[[(i,'Cancer')for i in ids1]]['Barcode']
c2 =  df1.loc[[(i,'Normal')for i in ids1]]['Barcode']
c3 =  df2.loc[[(i,'Myeloid cells')for i in ids1]]['Barcode']
c4 =  df2.loc[[(i,'T lymphocytes')for i in ids1]]['Barcode']
c5 =  df2.loc[[(i,'NK cells')for i in ids1]]['Barcode']
c6 =  df2.loc[[(i,'B lymphocytes')for i in ids1]]['Barcode']
c7 =  df2.loc[[(i,'Fibroblasts')for i in ids1]]['Barcode']

c1.index = ids1
c2.index = ids1
c3.index = ids1
c4.index = ids1
c5.index = ids1
c6.index = ids1
c7.index = ids1

df = pd.concat([c1,c2,c3,c4,c5,c6,c7],axis=1)
df.columns = ['Cancer','Normal','Myeloid cells',
 'T lymphocytes',
 'NK cells',
 'B lymphocytes',
 'Fibroblasts']

df.corr()

,Cancer,Normal,Myeloid cells,T lymphocytes,NK cells,B lymphocytes,Fibroblasts
Cancer,1.000000,-0.515495,-0.478591,-0.348057,-0.563558,-0.208139,-0.224570
Normal,-0.515495,1.000000,0.896205,0.300753,0.986636,-0.472061,0.656800
Myeloid cells,-0.478591,0.896205,1.000000,0.572188,0.911299,-0.099655,0.814287
T lymphocytes,-0.348057,0.300753,0.572188,1.000000,0.262489,0.443430,0.854678
NK cells,-0.563558,0.986636,0.911299,0.262489,1.000000,-0.388864,0.595738
B lymphocytes,-0.208139,-0.472061,-0.099655,0.443430,-0.388864,1.000000,-0.023689
Fibroblasts,-0.224570,0.656800,0.814287,0.854678,0.595738,-0.023689,1.000000


In [37]:
info = pd.read_table('../GSE131907_Lung_Cancer_cell_annotation.txt',index_col=0)
predict = pd.read_csv('../merge_data_h5_file/merge_data_predict_Integration_.csv',index_col=0)
info.index.name = None
ids = info[info['Cell_type'] == 'Epithelial cells'].index.intersection(predict.index)
predict = predict.loc[ids]
info.loc[info['Cell_type'] == 'Epithelial cells','Cell_type'] = predict['scale_predict']
#info.merge(predict)
ids = list(Counter(info['Sample']).keys())
names = globals()
for i in ids:
    names[i] = info.loc[info['Sample'] == i].groupby(['Cell_type']).count()['Barcode']
    
sample_cells = pd.concat([
    LUNG_N01, LUNG_N06, LUNG_N08, LUNG_N09, LUNG_N18, LUNG_N19, LUNG_N20,
    LUNG_N28, LUNG_N30, LUNG_N31, LUNG_N34, LUNG_T06, LUNG_T08, LUNG_T09,
    LUNG_T18, LUNG_T19, LUNG_T20, LUNG_T25, LUNG_T28, LUNG_T30, LUNG_T31,
    LUNG_T34, BRONCHO_58, EBUS_06, EBUS_28, EBUS_49, LN_01, LN_02, LN_03,
    LN_04, LN_05, LN_06, LN_07, LN_08, LN_11, LN_12, BRONCHO_11, EBUS_10,
    EBUS_12, EBUS_13, EBUS_15, EBUS_19, EBUS_51, EFFUSION_06, EFFUSION_11,
    EFFUSION_12, EFFUSION_13, EFFUSION_64, NS_02, NS_03, NS_04, NS_06, NS_07,
    NS_12, NS_13, NS_16, NS_17, NS_19
],axis=1).fillna(0)
sample_cells.columns = ids

sample_cells = sample_cells.T[['B lymphocytes', 'Cancer', 
       'MAST cells', 'Myeloid cells', 'NK cells', 'Normal', 'T lymphocytes']]
sample_cells.corr()

Cell_type,B lymphocytes,Cancer,MAST cells,Myeloid cells,NK cells,Normal,T lymphocytes
Cell_type,,,,,,,
B lymphocytes,1.000000,-0.272502,-0.105839,-0.332666,-0.371507,-0.414025,0.503700
Cancer,-0.272502,1.000000,-0.059909,-0.191699,-0.214695,0.099846,-0.461383
MAST cells,-0.105839,-0.059909,1.000000,0.296951,0.161177,0.319036,0.012609
Myeloid cells,-0.332666,-0.191699,0.296951,1.000000,0.526374,0.461373,-0.269651
NK cells,-0.371507,-0.214695,0.161177,0.526374,1.000000,0.633498,-0.158742
Normal,-0.414025,0.099846,0.319036,0.461373,0.633498,1.000000,-0.301928
T lymphocytes,0.503700,-0.461383,0.012609,-0.269651,-0.158742,-0.301928,1.000000


In [38]:
info = pd.read_table('../GSE131907_Lung_Cancer_cell_annotation.txt',index_col=0)
predict = pd.read_csv('../merge_data_h5_file/merge_data_predict_Integration_.csv',index_col=0)
info.index.name = None
ids = info[info['Cell_type'] == 'Epithelial cells'].index.intersection(predict.index)
predict = predict.loc[ids]
info.loc[info['Cell_type'] == 'Epithelial cells','Cell_type'] = predict['scale_predict']
#info.merge(predict)
ids = list(Counter(info['Sample_Origin']).keys())
names = globals()
for i in ids:
    names[i] = info.loc[info['Sample_Origin'] == i].groupby(['Cell_type']).count()['Barcode']
    
sample_cells = pd.concat([nLung, tLung, names['tL/B'], nLN, mLN, PE, mBrain],axis=1).fillna(0)
sample_cells.columns = ids
sample_cells = sample_cells.T[['B lymphocytes', 'Cancer', 
       'MAST cells', 'Myeloid cells', 'NK cells', 'Normal', 'T lymphocytes']]
sample_cells.corr()

Cell_type,B lymphocytes,Cancer,MAST cells,Myeloid cells,NK cells,Normal,T lymphocytes
Cell_type,,,,,,,
B lymphocytes,1.000000,-0.408882,-0.156611,-0.377764,-0.476994,-0.473614,0.776918
Cancer,-0.408882,1.000000,0.193079,-0.080356,-0.290515,-0.163622,-0.510476
MAST cells,-0.156611,0.193079,1.000000,0.705302,0.404738,0.604514,0.253705
Myeloid cells,-0.377764,-0.080356,0.705302,1.000000,0.903880,0.931329,-0.020643
NK cells,-0.476994,-0.290515,0.404738,0.903880,1.000000,0.949449,-0.095266
Normal,-0.473614,-0.163622,0.604514,0.931329,0.949449,1.000000,-0.029314
T lymphocytes,0.776918,-0.510476,0.253705,-0.020643,-0.095266,-0.029314,1.000000
